## [과제]구별로 가장 비싼, 가장 저렴한 주유소 각 1곳을 표시하기

In [2]:
import requests
from urllib.parse import quote
import numpy as np
import pandas as pd

### 1.데이터 리스트 가져오기 및 구분

In [4]:
#서울시 오일리스트 읽어오기
gu_oil =pd.read_csv('서울_주유(220418).csv') 
gu_oil.head()

,상호,주소,가격,셀프,상표,구
0,재건에너지 재정제2주유소 고속셀프지점,서울특별시 강동구 천호대로 1246 (둔촌제2동),1969,Y,현대오일뱅크,강동구
1,구천면주유소,서울 강동구 구천면로 357 (암사동),1983,N,현대오일뱅크,강동구
2,지에스칼텍스㈜ 신월주유소,서울 강동구 양재대로 1323 (성내동),1995,N,GS칼텍스,강동구
3,지에스칼텍스㈜ 동서울주유소,서울 강동구 천호대로 1456 (상일동),2015,Y,GS칼텍스,강동구
4,대성석유(주)길동주유소,서울 강동구 천호대로 1168,2025,N,GS칼텍스,강동구


In [5]:
#오류나는 주소 삭제
gu = gu_oil[~gu_oil['주소'].str.contains('대흥로 61')]

In [6]:
#구별 제일 비싼곳 리스트
df1 = gu.groupby('구').max()
df1.head(3)

,상호,주소,가격,셀프,상표
구,,,,,
강남구,현대오일뱅크㈜직영 삼성동주유소,서울 강남구 헌릉로 730,2548,Y,현대오일뱅크
강동구,현대오일뱅크㈜직영 명일셀프주유소,서울특별시 강동구 천호대로 1246 (둔촌제2동),2298,Y,현대오일뱅크
강북구,현대오일뱅크㈜직영 새한셀프주유소,서울 강북구 한천로 963 (번동),2039,Y,현대오일뱅크


In [7]:
#구별 제일 싼 곳 리스트
df2 = gu.groupby('구').min()
df2.head(3)

,상호,주소,가격,셀프,상표
구,,,,,
강남구,(유)동하석유 힐탑셀프주유소,서울 강남구 남부순환로 2718 (도곡2동),1977,N,GS칼텍스
강동구,(주)삼표에너지 고덕주유소,서울 강동구 상일로 132 (강일동),1969,N,GS칼텍스
강북구,(주)서울에너지 시민주유소,서울 강북구 덕릉로 158 (번동),1934,N,GS칼텍스


### 2.카카오 API 활용 위도 경도 불러오기

In [3]:
with open('../04.지도시각화/kakaoapikey.txt') as file:
    kakao_key = file.read()

len(kakao_key)

32

In [8]:
local_url = "https://dapi.kakao.com/v2/local/search/address.json"
header = {'Authorization' : f'KakaoAK {kakao_key}'}

In [15]:
lng_list1, lat_list1 = [], []
for i in df1.index:
    url = f'{local_url}?query={quote(df1.주소[i])}'
    result = requests.get(url, headers=header).json()
    lng_list1.append(float(result['documents'][0]['x']))
    lat_list1.append(float(result['documents'][0]['y']))
df1['위도'] = lat_list1
df1['경도'] = lng_list1
df1.head(3)

,상호,주소,가격,셀프,상표,위도,경도
구,,,,,,,
강남구,현대오일뱅크㈜직영 삼성동주유소,서울 강남구 헌릉로 730,2548,Y,현대오일뱅크,37.466557,127.119065
강동구,현대오일뱅크㈜직영 명일셀프주유소,서울특별시 강동구 천호대로 1246 (둔촌제2동),2298,Y,현대오일뱅크,37.536417,127.149372
강북구,현대오일뱅크㈜직영 새한셀프주유소,서울 강북구 한천로 963 (번동),2039,Y,현대오일뱅크,37.635564,127.032887


In [16]:
lng_list2, lat_list2 = [], []
for i in df2.index:
    url = f'{local_url}?query={quote(df2.주소[i])}'
    result = requests.get(url, headers=header).json()
    try:
        lng_list2.append(float(result['documents'][0]['x']))
        lat_list2.append(float(result['documents'][0]['y']))
    except:
        print(df2.주소[i])
        print(i)
df2['위도'] = lat_list2
df2['경도'] = lng_list2
df2.head(3)

,상호,주소,가격,셀프,상표,위도,경도
구,,,,,,,
강남구,(유)동하석유 힐탑셀프주유소,서울 강남구 남부순환로 2718 (도곡2동),1977,N,GS칼텍스,37.485415,127.043319
강동구,(주)삼표에너지 고덕주유소,서울 강동구 상일로 132 (강일동),1969,N,GS칼텍스,37.558767,127.171391
강북구,(주)서울에너지 시민주유소,서울 강북구 덕릉로 158 (번동),1934,N,GS칼텍스,37.635016,127.031310


### 3.지도위에 표시하기

In [13]:
import folium 

In [14]:
map = folium.Map(location=[37.559868, 126.967109] , zoom_start=14)
for i in df1.index:
    folium.Marker(
        location=[df1.위도[i], df1.경도[i]],
        popup=folium.Popup(df1.상호[i], max_width=200),
        tooltip=df1.상호[i],
        icon=folium.Icon(color='red', icon='glyphicon glyphicon-bell')
    ).add_to(map)

for i in df2.index:
    folium.Marker(
        location=[df2.위도[i], df2.경도[i]],
        popup=folium.Popup(df2.상호[i], max_width=200),
        tooltip=df2.상호[i],
        icon=folium.Icon(color='blue', icon='glyphicon glyphicon-remove-sign')
    ).add_to(map)

title_html = '<h3 align="center" style="font-size:20px">구별 가장 싼/비싼 주유소</h3>'    
map.get_root().html.add_child(folium.Element(title_html))
map